In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/logistic

/content/drive/MyDrive/logistic


In [3]:
!pwd

/content/drive/MyDrive/logistic


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as mpl

In [5]:
df_train = pd.read_csv('/content/drive/MyDrive/logistic/emnist-letters-train.csv', header= None)
train = df_train.to_numpy()

In [6]:
input1 = train[0:62000, 1:]/255 #Scaling values between 0 and 1
ylabels = train[0:62000, 0]

In [7]:
x0 = np.ones([np.shape(input1)[0], 1])
input = np.concatenate((x0, input1),axis = 1)
print(np.shape(input))

(62000, 785)


In [8]:
uniquech = set(ylabels)
logisticY = np.zeros([np.size(ylabels), len(uniquech)])
for i in range(np.size(ylabels)):
  for j in range(1, len(uniquech)+1):
    if ylabels[i] == j:
      logisticY[i,j-1] = 1
print(logisticY)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


In [9]:
def sigmoid(z):
  return 1/ (1+np.exp(-z))

In [24]:
def accuracy(label, preds):
  truthtable = (label == preds).astype(int)
  truthlist = truthtable.tolist()
  accuracy = (truthlist.count([1]))*(100/(np.shape(preds)[0]))
  print("Accuracy is", accuracy , "%" )
  return accuracy

In [10]:
def train(x, y, alpha, iterations):

#initialise thetas
  w1 = np.random.random([x.shape[1], 100]) - 0.5
  w2 = np.random.random([100, len(uniquech)]) - 0.5
  b1 = np.zeros([1, 100], dtype = int)
  b2 = np.zeros([1, len(uniquech)], dtype = int)
  for iter in range(iterations):

#forward
    z2 = np.dot(x, w1) + b1
    a2 = sigmoid(z2)
    z3 = np.dot(a2, w2) + b2
    out = sigmoid(z3)
    
#errors
    d3 = (out - y)
    d2 = (np.dot(d3 ,w2.T)) * (a2*(1 - a2))

#backward
    pd_w2 = np.dot(a2.T, d3) / (x.shape[0])
    pd_w1 = np.dot(x.T , d2) / (x.shape[0])
    pd_b2 = np.sum(d3) / (x.shape[0])
    pd_b1 = np.sum(d2) / (x.shape[0])
    w1 = w1 - (alpha*pd_w1) 
    w2 = w2 - (alpha*pd_w2) 
    b1 = b1 - (alpha*pd_b1)
    b2 = b2 - (alpha*pd_b2)

#cost function
    cost = (1/(x.shape[1])) * np.sum((-y*np.log(out)) - ((1-y) *np.log(1 - out)))
    if( iter % (iterations/10) == 0):
      print("Cost after", iter , "is :", cost)
    
  return w1, w2, out

In [11]:
w1, w2, out = train(x=input, y=logisticY, alpha = 0.3, iterations= 1000)

Cost after 0 is : 1977.9382522985204
Cost after 100 is : 235.12783254323708
Cost after 200 is : 196.34446508661938
Cost after 300 is : 178.9441127827795
Cost after 400 is : 168.23329506385545
Cost after 500 is : 160.50142248036008
Cost after 600 is : 154.42917608115707
Cost after 700 is : 149.4069974379171
Cost after 800 is : 145.1047926563711
Cost after 900 is : 141.32596062917096


**TEST**

In [25]:
df_test = pd.read_csv('/content/drive/MyDrive/logistic/emnist-letters-test.csv', header= None)
test = df_test.to_numpy()

In [17]:
testX1 = test[:, 1:]/255
testY = test[:, 0]

In [18]:
x0t = np.ones([np.shape(testX1)[0], 1])
testX = np.concatenate((x0t, testX1),axis = 1)

In [19]:
a2 = sigmoid(np.dot(testX, w1))
outtest = sigmoid(np.dot(a2, w2))

In [20]:
indext = np.argmax(outtest, axis=1)
predt = np.zeros(outtest.shape)
for i in range(np.shape(outtest)[0]):
  predt[i][indext[i]] = 1

In [21]:
predictiont = np.zeros([np.size(testY) , 1], dtype = int)
for i in range(np.shape(outtest)[0]):
  for j in range(np.shape(outtest)[1]):
    if predt[i][j] == 1:
      predictiont[i][0] = j + 1

In [23]:
accuracy(np.vstack(testY) , predictiont)

Accuracy is 67.88513513513514 %


67.88513513513514